<h1><center> Market Basket Analysis on UK Online Retail Dataset </h1></center>

# Table of contents

* [Business Problem](#bproblem)
* [EDA](#eda)
* [FP-growth algorithm: creating rules](#fpgrowth)
* [Sort and display the rules](#display)

<h2> Business Problem </h2> <a name="bproblem"></a>

In this project we are going to use a Market basket analysis to **gain insights into granular behavior of customers**.  
  
Let's imagine a UK-based online retail store that collected data from customers transactions wanted to use it in order to increase sales. Suppose they ask us: Can we uncover trends or patterns in decisions taken by our customers? The answer is: Yes, you can! That's what Market Basket Analysis (MBA) and Association rules is all about!  
  
What do we mean by **Association rules**?  
Association rules is one of the key concepts of machine learning used in market basket analysis. These rules will allow us to find products that are frequently bought together. Algorithms like _Apriori_ and _FP-growth_ do not extract individual preferences, but find relationships between items from huge databases.
  
But, what exactly is an association rule? It consists of an **antecedent** and a **consequent**, both of which are a list of items. For example, an association rule in a grocery store dataset could be the following:  
  
                                      {Bread, Egg -> Butter}
  
Various metrics are used to help us understand the **strength of association** between the antedecent (in this case _Bread, Egg_) and the consequent (in this case _Butter_). If these metrics are high enough, we could say the probability of having _Butter_ on the cart with the knowledge that _Bread_ and _Egg_ are present is higher than if the cart does not contain Bread and Egg. Therefore, there is a cross-selling opportunity between the antecedent and the consequent.  
  
By knowing which products tend to be bought together, the retailer can offer combinations of products (**product bundling**) or run a marketing scheme that offers a combination of discounts.
  
The aim of our project is to generate association rules from our transactional data. The dataset is from **UCI Machine Learning Repository** (https://archive.ics.uci.edu/ml/datasets/online+retail).

<h2> EDA </h2> <a name="eda"></a>

First, let's import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np

# orange3-associate package contains the implementation of FP growth 
# provided by the group who developed for the Orange data mining package
import Orange
from Orange.data import Domain, DiscreteVariable, ContinuousVariable
from orangecontrib.associate.fpgrowth import *

import matplotlib.pyplot as plt
%matplotlib inline

Our transactions dataset is an excel file located in our current working directory. Let's convert it to a pandas dataframe.

In [2]:
trans_df = pd.read_excel('Online_Retail.xlsx')
trans_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


The dataset contains the following information:  
  
* **InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
  
  
* **StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. 
  
  
* **Description**: Product (item) name. Nominal.
  
  
* **Quantity**: The quantities of each product (item) per transaction. Numeric.	  
  
  
* **InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated. 
  
   
* **UnitPrice**: Unit price. Numeric, Product price per unit in sterling.  
  
  
* **CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
  
  
* **Country**: Country name. Nominal, the name of the country where each customer resides.

In [3]:
desc = trans_df.describe().T
desc['missing %'] = 1 - (desc['count'] / len(trans_df))
desc

,count,mean,std,min,25%,50%,75%,max,missing %
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0,0.000000
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0,0.000000
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0,0.249267


Almost 25% of transactions do not have a CustomerID associated. As we are not using this variable for our analysis we are going to keep them. We are not going to delete cancelation neither.
  
However, we can see there are negative values for **Quantity** and **UnitPrice**. They might be returns or errors, therefore we are not going to include them in our analysis. 

In [4]:
trans_df = trans_df[~(trans_df['Quantity']<0)]
trans_df = trans_df[~(trans_df['UnitPrice']<0)]
print(trans_df.shape)
trans_df.head()

(531283, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
countries_df = trans_df['Country'].value_counts().to_frame().reset_index()
countries_df.rename(columns={'index':'country', 'Country':'count'}, inplace=True)
countries_df['country_perc'] = (countries_df['count'] / countries_df['count'].sum()) * 100
countries_df.head()

,country,count,country_perc
0,United Kingdom,486284,91.530126
1,Germany,9042,1.701918
2,France,8408,1.582584
3,EIRE,7894,1.485837
4,Spain,2485,0.467736


More than 91% of the transactions are from UK, we are going to focus on the largest segment.

In [6]:
trans_df = trans_df[trans_df['Country'] == 'United Kingdom']

print(trans_df.shape)
trans_df.head()

(486284, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
print('There are {:,} unique transactions.'.format(len(trans_df.InvoiceNo.unique())))
print('\nThere are {:,} unique items.'.format(len(trans_df.Description.unique())))

There are 18,784 unique transactions.

There are 4,059 unique items.


Now that we know that, we need to build a transactions matrix where each of the 18,784 rows will represent a transaction and the 4,059 columns will represent the product (<b>0</b>: the transaction does not contain this product, <b>1</b>: the transaction does contain this item).  
  
First, let's group our transactions and list the products for each one of them:

In [8]:
grouped_df = trans_df[['InvoiceNo', 'Description']].groupby('InvoiceNo')['Description'].apply(list)
grouped_df = grouped_df.to_frame()
grouped_df.head()

,Description
InvoiceNo,
536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
536369,[BATH BUILDING BLOCK WORD]


Now, we will get a list of all these items and build a dictionary with each item having a value of 0.

In [9]:
items = list(trans_df.Description.unique())
print(len(items))

4059


In [10]:
transaction_dict = {item:0 for item in items}

Now we can build the transactions matrix:

In [11]:
output_list = list()
for index, row in grouped_df.iterrows():
    products = row['Description']
    row_val = {item:0 for item in items}
    row_val.update({item:1 for item in products})
    output_list.append(row_val)

items_df = pd.DataFrame(output_list)
print(items_df.shape)
items_df.head()

(18784, 4059)


,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,5 HOOK HANGER MAGIC TOADSTOOL,60 TEATIME FAIRY CAKE CASES,"AIRLINE LOUNGE,METAL SIGN",ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE RED,...,SET OF 6 RIBBONS COUNTRY STYLE,SNACK TRAY RED VINTAGE DOILY,SET OF 6 RIBBONS PERFECTLY PRETTY,SET OF 6 RIBBONS PARTY,SET 10 CARDS SNOWY ROBIN 17099,SET 10 CARDS SWIRLY XMAS TREE 17104,"LETTER ""U"" BLING KEY RING",CREAM HANGING HEART T-LIGHT HOLDER,BLACK SIL'T SQU CANDLE PLATE,"PAPER CRAFT , LITTLE BIRDIE"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's check what are the top 20 items that occur in the dataset and how much of the total sales they account for.

In [12]:
top_items = items_df.sum()
top_items = top_items.to_frame()
top_items.rename(columns={'index':'item_name', 0:'item_count'}, inplace=True)
top_items = top_items.sort_values(by='item_count', ascending=False).reset_index()
top_items.head()

,index,item_count
0,WHITE HANGING HEART T-LIGHT HOLDER,2166
1,JUMBO BAG RED RETROSPOT,1938
2,REGENCY CAKESTAND 3 TIER,1685
3,PARTY BUNTING,1594
4,LUNCH BAG RED RETROSPOT,1392


In [13]:
top_items['item_perc'] = top_items['item_count'] / top_items['item_count'].sum()
top_items['cumsum'] = top_items['item_perc'].cumsum()
top_items.head(20)

,index,item_count,item_perc,cumsum
0,WHITE HANGING HEART T-LIGHT HOLDER,2166,0.004553,0.004553
1,JUMBO BAG RED RETROSPOT,1938,0.004073,0.008626
2,REGENCY CAKESTAND 3 TIER,1685,0.003542,0.012167
3,PARTY BUNTING,1594,0.003350,0.015518
4,LUNCH BAG RED RETROSPOT,1392,0.002926,0.018444
5,ASSORTED COLOUR BIRD ORNAMENT,1371,0.002882,0.021325
6,SET OF 3 CAKE TINS PANTRY DESIGN,1241,0.002608,0.023933
7,NATURAL SLATE HEART CHALKBOARD,1219,0.002562,0.026496
8,LUNCH BAG BLACK SKULL.,1216,0.002556,0.029051
9,HEART OF WICKER SMALL,1164,0.002447,0.031498


The top 20 items that are bought represent more or less 5% of the sales, which is not that much. 

We absolutely need to prune our dataset, as generating all association rules for the entire set of items would be very unefficient and computing time would be huge. 

For example, the following code mines for patterns on the **top 15** most sold products. We'll use the <i>prune_dataset</i> function that we defined bellow.

In [14]:
def prune_dataset(input_df, length_trans = 2, total_sales_perc = 0.5, start_item = None, end_item = None):
    if 'total_items' in input_df.columns:
        del(input_df['total_items'])
    item_count = input_df.sum().sort_values(ascending = False).reset_index()
    total_items = sum(input_df.sum().sort_values(ascending = False))
    item_count.rename(columns={item_count.columns[0]:'item_name',item_count.columns[1]:'item_count'}, inplace=True)
    if not start_item and not end_item: 
        item_count['item_perc'] = item_count['item_count']/total_items
        item_count['total_perc'] = item_count.item_perc.cumsum()
        selected_items = list(item_count[item_count.total_perc < total_sales_perc].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items], item_count[item_count.total_perc < total_sales_perc]
    elif end_item > start_item:
        selected_items = list(item_count[start_item:end_item].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items],item_count[start_item:end_item]

The following code will help us select only that subset of data containing the top 15 items:

In [15]:
output_df_uk, item_counts_uk = prune_dataset(input_df=items_df, length_trans=2, start_item=0, end_item=15)
print(output_df_uk.shape)
output_df_uk.head()

(4961, 15)


,WHITE HANGING HEART T-LIGHT HOLDER,JUMBO BAG RED RETROSPOT,REGENCY CAKESTAND 3 TIER,PARTY BUNTING,LUNCH BAG RED RETROSPOT,ASSORTED COLOUR BIRD ORNAMENT,SET OF 3 CAKE TINS PANTRY DESIGN,NATURAL SLATE HEART CHALKBOARD,LUNCH BAG BLACK SKULL.,HEART OF WICKER SMALL,JUMBO BAG PINK POLKADOT,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SUKI,PACK OF 72 RETROSPOT CAKE CASES,PAPER CHAIN KIT 50'S CHRISTMAS
12,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
14,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
16,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
21,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


4,961 transactions have those items along with other items.  
  
The next step is to convert this selected data into the required _Orange_ Table data structure.

In [16]:
input_assoc_rules = output_df_uk
domain_retail = Domain([DiscreteVariable.make(name=item, values=['0', '1']) for item in input_assoc_rules.columns])
data_retail_1 = Orange.data.Table.from_numpy(domain=domain_retail, X=input_assoc_rules.values, Y=None)
data_retail_1_en, mapping = OneHot.encode(data_retail_1, include_class=False)

In [17]:
print(data_retail_1_en.shape)

(4961, 30)


It makes sense, every item is duplicated ('0' or '1'). 15 x 2 = 30.  
  
Now the final step is creating our rules. We will specify two pieces of information for generating our rules: **support** <i>(min_support = 0.01, which means required transactions = 49)</i> and **confidence** <i>(confidence = 0.3)</i>. 

<h2> FP-growth algorithm: creating rules </h2> <a name="fpgrowth"></a>

In [18]:
# set min support and confidence
min_support = 0.01
confidence = 0.3

print('Number of required transactions = ', int(input_assoc_rules.shape[0]*min_support))

Number of required transactions =  49


In [19]:
itemsets = dict(frequent_itemsets(data_retail_1_en, min_support=min_support))
print('Number of itemsets we get for a support of 1% = ', len(itemsets))

Number of itemsets we get for a support of 1% =  645632


We get **645,632** frequent itemsets for a support of only 1%!

In [20]:
rules_df = pd.DataFrame()
if len(itemsets) < 1000000: 
    rules = [(P, Q, supp, conf)
    for P, Q, supp, conf in association_rules(itemsets, confidence)
       if len(Q) == 1 ]

    names = {item: '{}={}'.format(var.name, val)
        for item, var, val in OneHot.decode(mapping, data_retail_1, mapping)}
    
    eligible_ante = [v for k,v in names.items() if v.endswith("1")]
    
    N = input_assoc_rules.shape[0]
    
    rule_stats = list(rules_stats(rules, itemsets, N))
    
    rule_list_df = []
    for ex_rule_frm_rule_stat in rule_stats:
        ante = ex_rule_frm_rule_stat[0]            
        cons = ex_rule_frm_rule_stat[1]
        named_cons = names[next(iter(cons))]
        if named_cons in eligible_ante:
            rule_lhs = [names[i][:-2] for i in ante if names[i] in eligible_ante]
            ante_rule = ', '.join(rule_lhs)
            if ante_rule and len(rule_lhs)>1 :
                rule_dict = {'support' : ex_rule_frm_rule_stat[2],
                             'confidence' : ex_rule_frm_rule_stat[3],
                             'coverage' : ex_rule_frm_rule_stat[4],
                             'strength' : ex_rule_frm_rule_stat[5],
                             'lift' : ex_rule_frm_rule_stat[6],
                             'leverage' : ex_rule_frm_rule_stat[7],
                             'antecedent': ante_rule,
                             'consequent':named_cons[:-2] }
                rule_list_df.append(rule_dict)
    rules_df = pd.DataFrame(rule_list_df)
    print("Raw rules data frame of {} rules generated".format(rules_df.shape[0]))
    if not rules_df.empty:
        pruned_rules_df = rules_df.groupby(['antecedent','consequent']).max().reset_index()
    else:
        print("Unable to generate any rule")

Raw rules data frame of 117464 rules generated


<h2> Sort and display the rules </h2> <a name="display"></a>

In [21]:
dw = pd.options.display.max_colwidth
pd.options.display.max_colwidth = 100
(pruned_rules_df[['antecedent','consequent',
                  'support','confidence','lift']].groupby('consequent')
                                                 .max()
                                                 .reset_index()
                                                 .sort_values(['lift', 'support','confidence'],
                                                              ascending=False)).head(5)

,consequent,antecedent,support,confidence,lift
8,PACK OF 72 RETROSPOT CAKE CASES,"WHITE HANGING HEART T-LIGHT HOLDER, REGENCY CAKESTAND 3 TIER, NATURAL SLATE HEART CHALKBOARD",145,0.971014,5.394404
9,PAPER CHAIN KIT 50'S CHRISTMAS,"WHITE HANGING HEART T-LIGHT HOLDER, REGENCY CAKESTAND 3 TIER, NATURAL SLATE HEART CHALKBOARD",94,0.597701,4.341428
3,JUMBO SHOPPER VINTAGE RED PAISLEY,"WHITE HANGING HEART T-LIGHT HOLDER, PAPER CHAIN KIT 50'S CHRISTMAS",384,0.879310,4.218819
5,LUNCH BAG BLACK SKULL.,"WHITE HANGING HEART T-LIGHT HOLDER, PACK OF 72 RETROSPOT CAKE CASES, LUNCH BAG RED RETROSPOT",227,0.852459,4.078157
4,JUMBO STORAGE BAG SUKI,"WHITE HANGING HEART T-LIGHT HOLDER, SET OF 3 CAKE TINS PANTRY DESIGN , JUMBO BAG PINK POLKADOT",405,0.852459,4.016191


#### How to read this dataframe?  
  
If we take the first rule, we can say: people who bought _WHITE HANGING HEART T-LIGHT HOLDER, REGENCY CAKESTAND 3 TIER and NATURAL SLATE HEART CHALKBOARD_ also tend to buy _PACK OF 72 RETROSPOT CAKE CASES_.  

* **Support** of the rule is **145**,which means all the items together appear in 145 transactions in the dataset. 
  
    
    
* **Confidence = 97%**, which means that 97% of the time the antecedent items occurred we also had the consequent in the transaction. 

  
  
* **Lift = 5.39** means that the probability of finding a _PACK OF 72 RETROSPOT CAKE CASES_ in the transactions which have _WHITE HANGING HEART T-LIGHT HOLDER, REGENCY CAKESTAND 3 TIER and NATURAL SLATE HEART CHALKBOARD_ is greater than the normal probability of finding a _PACK OF 72 RETROSPOT CAKE CASES_ in the previous transactions. A value greater than 1 indicates it is a very good quality rule.